In [15]:
import sys
print("SYS",sys.version)
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)

# tf.set_random_seed(42)
import keras.backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Activation, Dropout, RepeatVector
from keras.utils import np_utils
# from tf.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle

SYS 3.10.2 (main, Mar 16 2022, 10:43:30) [GCC 4.8.5 20150623 (Red Hat 4.8.5-44)]
2.10.0


In [18]:
model_file = "LSTM_embeddinglayer_dropout50_epochs70_final.dat"
mf=open(model_file,'rb')
model_pred = pickle.load(mf)

In [19]:
# This is the training data
#training_file = sys.argv[1]
training_file ="train.tsv"
# This is the filename that we will save the model in
#model_file = sys.argv[2]


corpus = ""
for line in open(training_file).readlines():

	# Each line in the training data is two columns
	# тыкгаткэта ивнин риӄукэтэ	ты>кгат>кэ>та ив>ни>н риӄукэ>тэ
	# print("line",line)
	row = line.strip().split('\t')
	# print("row", row)
	# The tokens are in the first column
	tokens = row[0]
	# print(tokens)
	# print("tokens", tokens)
	# Add the tokens to the list of sentences, with a beginning of sentence and an end of sentence marker
	# if len(corpus)==0:

	corpus+=tokens
	# else:
	# 	corpus.extend(tokens)

print(set(corpus))
chars = sorted(list(set(corpus)))
print(chars)
char_index = dict((c,i) for i,c in enumerate(chars))
index_char = dict((i,c) for i,c in enumerate(chars))
char_index["<UNK>"]=len(chars)
index_char[len(chars)]="<UNK>"
print(char_index)
print(index_char)

SEQUENCE_LENGTH = 10

{'ӈ', 'ԓ', 'щ', '’', 'ң', ' ', 'о', 'ю', 'р', 'ы', 'у', 'ш', 'й', 'ч', 'с', 'ё', 'з', 'г', 'в', 'ь', 'д', 'б', 'э', 'ц', 'ʼ', 'к', 'ж', 'ӄ', 'қ', 'п', 'т', 'и', 'ъ', 'е', 'х', 'ф', 'н', 'я', 'а', 'м'}
[' ', 'ʼ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'қ', 'ң', 'ӄ', 'ӈ', 'ԓ', '’']
{' ': 0, 'ʼ': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'м': 13, 'н': 14, 'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19, 'у': 20, 'ф': 21, 'х': 22, 'ц': 23, 'ч': 24, 'ш': 25, 'щ': 26, 'ъ': 27, 'ы': 28, 'ь': 29, 'э': 30, 'ю': 31, 'я': 32, 'ё': 33, 'қ': 34, 'ң': 35, 'ӄ': 36, 'ӈ': 37, 'ԓ': 38, '’': 39, '<UNK>': 40}
{0: ' ', 1: 'ʼ', 2: 'а', 3: 'б', 4: 'в', 5: 'г', 6: 'д', 7: 'е', 8: 'ж', 9: 'з', 10: 'и', 11: 'й', 12: 'к', 13: 'м', 14: 'н', 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф', 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 

In [20]:
# Generates the sequence similar to above methods. Gets the generated string using the model.
def predict_next_n_chars(pattern, n):
    #print("pattern in pred next",pattern)
    output=""
    for i in range(n):
      x = np.reshape(pattern, (1, len(pattern), 1))
      #print("x",x.shape)
      #prediction = model.predict(x, verbose=0)
      prediction = model_pred.predict(x, verbose=0)
      #print (index_char[np.argmax(prediction)], end = '') #get next char index.
      output+=index_char[np.argmax(prediction)]
      seq_in = [index_char.get(value,"<UNK>") for value in pattern]
      pattern.append(np.argmax(prediction))
      pattern = pattern[1:len(pattern)]
      if index_char[np.argmax(prediction)]==" ":
            break
            
    return output

In [21]:
def prepare_input(pattern):
    if len(pattern)>SEQUENCE_LENGTH:
      #print("test_string[i] and i",test_string[i], i)
      #print("in if")
      pattern=pattern[-10:]
      input_str =[char_index.get(value,40) for value in pattern]
    else:
      #print("test_string[i] and i",test_string[i], i)
      input_str =[char_index.get(value,40) for value in pattern]
      for j in range(len(input_str),SEQUENCE_LENGTH):
        input_str.append(0)
        
    return input_str

In [22]:
def predictive_text2(test_string):
    hits=0
    tokens=0
    clicks=0
#     print("test string:", test_string)
#     print("test string -2",test_string[-2])
#     print(len(test_string))
    words=test_string.split(" ")
   # print("words",words)
    output = []
    tokens=len(words)
    pattern =''
    for word in words:
        #print("word is:",word)
        if words[0]!=word:
            pattern+=" "
        for i in range(len(word)):
            three_preds=[]
            pattern+=word[i]
            input_str=prepare_input(pattern)
            #print("Input",input_str)
            #print("Pattern",pattern)
            x = np.reshape(input_str, (1, len(input_str), 1))
            #print("x",x.shape)
            pred=model_pred.predict(x)
            #print(pred)
            first=index_char.get(np.argmax(pred),"<UNK>")
            second = index_char.get(np.argsort(np.max(pred,axis=0))[-2],"<UNK>")
            third = index_char.get(np.argsort(np.max(pred,axis=0))[-3],"<UNK>")
#             print(first)
#             print(second)
#             print(third)
            first_word = predict_next_n_chars(prepare_input(pattern+first),20)
            second_word = predict_next_n_chars(prepare_input(pattern+second),20)
            third_word = predict_next_n_chars(prepare_input(pattern+third),20)
            first_pred = first+first_word
            second_pred = second+second_word
            third_pred = third+third_word
            actual=word[i+1:]
#             print("first",first_pred)
#             print("second",second_pred)
#             print("third",third_pred)
            #three_preds=[first_pred,second_pred,third_pred]
            
#             print("to be predicted",actual)
            if (actual).strip()== first_pred.strip() or (actual).strip()==second_pred.strip() or (actual).strip()==third_pred.split():
                #print("in hits")
                output.append(actual.strip())
                output.append(" ")
                hits+=1
                clicks+=1
                break
            elif i<len(word)-1:
                output.append(word[i+1])
                clicks+=1
            else:
                output.append(" ")
                clicks+=1

    #print("hits",hits)
    return [clicks,tokens,hits,[test_string,output]]


In [23]:
validate=open('test.tsv','r').readlines()
validation_data=[]
for line in validate:
    lines=line.split('\t')
    text=lines[0]
    validation_data.append(text)

In [24]:
final_result=[]
total_hits=0
total_clicks=0
total_tokens=0
output_file2 = open("output_seq10_epochs70_test_200.txt","a")
for line in validation_data[:200]:
    clicks,tokens,hits,outputs=predictive_text2(line)
    line,output=outputs
    output_file2.write(line+"   "+str(output)+"\n")
    #print(output)
    #print("hits",hits)
    final_result.append(outputs)
    total_hits+=hits
    total_clicks+=clicks
    total_tokens+=tokens

1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 39ms/step


...

1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 42ms/step


In [25]:
print(total_hits)
print(total_clicks)
print(total_tokens)
print(total_clicks/total_tokens)

353
6034
846
7.132387706855792
